In [1]:
import warnings
import pprint
import skrebate
import imblearn
import time
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve, GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix, roc_curve
from sklearn import metrics
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
from mlxtend.regressor import StackingCVRegressor, StackingRegressor
from mlxtend.classifier import StackingCVClassifier, StackingClassifier

In [4]:
Galaxy_ML = os.path.dirname(os.path.dirname(os.getcwd()))

sys.path.append(os.path.join(Galaxy_ML, 'galaxy_ml'))
try:
    from iraps_classifier import BinarizeTargetClassifier, BinarizeTargetRegressor
except ImportError:
    raise ImportError("galaxy_ml is not in path!")



In [5]:
Afatinib_1032_2019 = pd.read_csv(os.path.join(Galaxy_ML, 'galaxy_ml/test-data/Afatinib_1032.tsv.gz'), sep='\t', index_col=0)
Afatinib_1032_2019

,depmap_ID,DRUG_ID,DRUG_NAME,Primary Disease,Gender,AUC,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),...,NUDT3 (ENSG00000272325),C6ORF165 (ENSG00000272514),MUSTN1 (ENSG00000272573),DOC2B (ENSG00000272636),RP11-309L24.9 (ENSG00000272899),C2ORF15 (ENSG00000273045),GRIN2B (ENSG00000273079),SNURF (ENSG00000273173),RP11-1263C18.1 (ENSG00000273238),ZBTB8B (ENSG00000273274)
5,ACH-000087,1032,Afatinib,Bone Cancer,Male,0.979983,4.955127,0.070389,5.873567,2.853996,...,4.694880,0.137504,0.545968,3.203201,0.978196,0.669027,0.000000,7.092229,0.669027,0.042644
6,ACH-000327,1032,Afatinib,Lung Cancer,Female,0.976187,4.041769,0.000000,5.813525,1.918386,...,3.730096,0.028569,0.070389,0.000000,1.035624,2.310340,0.000000,0.641546,0.000000,0.000000
7,ACH-000905,1032,Afatinib,Bladder Cancer,Male,0.901265,5.881175,0.000000,6.643135,2.039138,...,2.664483,0.084064,0.226509,0.000000,0.422233,2.003602,0.000000,5.660210,0.000000,0.432959
8,ACH-000242,1032,Afatinib,Bladder Cancer,Male,0.987345,7.496574,0.000000,6.458940,2.364572,...,3.658783,0.739848,0.201634,0.000000,0.378512,3.557042,0.000000,6.843607,0.000000,0.948601
9,ACH-000384,1032,Afatinib,Bladder Cancer,Female,0.972656,5.755155,0.000000,6.928844,2.895303,...,3.324811,2.042644,1.531069,0.000000,0.622930,3.244887,0.000000,5.405312,0.000000,1.077243
13,ACH-000323,1032,Afatinib,Brain Cancer,Male,0.984639,4.590362,0.000000,6.647890,1.899176,...,3.187451,0.111031,0.526069,0.000000,0.298658,0.176323,0.000000,6.463524,0.650765,0.014355
14,ACH-000137,1032,Afatinib,Brain Cancer,Female,0.986041,5.108106,0.000000,6.888500,2.207893,...,3.861955,0.137504,0.879706,0.000000,0.333424,1.490570,0.056584,7.145372,0.000000,0.757023
15,ACH-000738,1032,Afatinib,Brain Cancer,Male,0.975092,4.078951,0.000000,6.691953,2.121015,...,3.056584,0.084064,0.855990,0.000000,0.536053,0.505891,0.014355,6.101818,0.000000,0.704872
16,ACH-000571,1032,Afatinib,Brain Cancer,Male,0.986202,3.408712,0.000000,6.677931,2.269033,...,3.247928,0.124328,0.000000,0.000000,0.731183,0.495695,0.000000,0.669027,0.000000,0.028569
17,ACH-000040,1032,Afatinib,Brain Cancer,Male,0.983713,3.828835,0.000000,6.436295,2.073820,...,3.782409,0.526069,0.516015,0.000000,1.021480,0.687061,0.014355,6.302685,0.000000,0.000000


In [6]:
X, y = Afatinib_1032_2019.iloc[:, 6:].values, Afatinib_1032_2019.iloc[:, 5].values.ravel()
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [7]:
# Shared testing parameters
from iraps_classifier import IRAPSCore, IRAPSClassifier
from iraps_classifier import binarize_auc_scorer, binarize_average_precision_scorer
from iraps_classifier import BinarizeTargetTransformer
from model_validations import OrderedKFold, RepeatedOrderedKFold

cv = RepeatedOrderedKFold(n_splits=5, n_repeats=2, random_state=123)

scoring = {
    'ap': binarize_average_precision_scorer,
    'auc': binarize_auc_scorer
}

# Tree algorithms

In [10]:
# RandomForestClassifier
model = ensemble.RandomForestClassifier(random_state=0, class_weight="balanced")
estimator = BinarizeTargetClassifier(model)

rval = rval_rfc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([1.35372734, 1.41321278, 1.29048705, 1.34246397, 1.38653302,
         1.40553212, 1.45303392, 1.46087599, 2.10531211, 2.00953484]),
  'score_time': array([0.01151896, 0.01190519, 0.01188588, 0.00988603, 0.01004791,
         0.00943494, 0.01052713, 0.02698493, 0.02202177, 0.02516317]),
  'test_ap': array([0.29785837, 0.20932054, 0.33723589, 0.46276211, 0.29384413,
         0.36180109, 0.35251422, 0.46318204, 0.33440364, 0.19001403]),
  'train_ap': array([0.99702381, 1.        , 0.99469955, 0.99947146, 1.        ,
         1.        , 1.        , 0.99148277, 1.        , 0.99792961]),
  'test_auc': array([0.75752688, 0.72134387, 0.84892473, 0.84032258, 0.76523477,
         0.84486166, 0.76935484, 0.79891304, 0.71075269, 0.71956522]),
  'train_auc': array([0.99970964, 1.        , 0.99958059, 0.9999684 , 1.        ,
         1.        , 1.        , 0.99911009, 1.        , 0.9998713 ])},
 0.3302936076023896,
 0.7776800280131276)

In [22]:
# RandomForestClassifier
from sklearn.model_selection import ShuffleSplit
model = ensemble.RandomForestClassifier(random_state=10, class_weight="balanced_subsample",
                                        criterion='gini', n_estimators=500, max_features='log2')
estimator = BinarizeTargetClassifier(model)
cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=123)

rval = rval_rfc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([4.15727186, 4.05841279, 3.47297621, 3.46279025, 3.11705279,
         3.21740007, 3.00927615, 3.15030193, 3.41917801, 3.60925078,
         3.08572888, 3.56215024, 3.00192094, 3.13034606, 3.07866311,
         3.96361804, 4.04161406, 3.73699403, 2.92310905, 2.79786873]),
  'score_time': array([0.13437819, 0.14243603, 0.10007501, 0.10168576, 0.10016608,
         0.10612893, 0.15090704, 0.11659503, 0.17390585, 0.10732794,
         0.15345311, 0.10576177, 0.13612509, 0.10640788, 0.11837506,
         0.29004908, 0.17708802, 0.09508109, 0.12139297, 0.09415698]),
  'test_ap': array([0.43389695, 0.28496133, 0.57994623, 0.68509468, 0.36172767,
         0.44607327, 0.33618534, 0.67570985, 0.4390107 , 0.42503526,
         0.3632379 , 0.39473922, 0.38696642, 0.48280744, 0.43056388,
         0.4458066 , 0.55090915, 0.49283605, 0.43074217, 0.36153392]),
  'train_ap': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.]),
  'test_auc': arr

In [20]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = SelectKBest(score_func=f_classif, k=300)
pipe = Pipeline([('kbest', kbest), ('est', ensemble.RandomForestClassifier(random_state=0))])

estimator = BinarizeTargetClassifier(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=2, random_state=123)
#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([1.00590611, 0.99522614, 0.98419499, 0.98422813, 0.96921015,
         0.97119427, 0.97762179, 0.97783518, 1.17131901, 1.15477991]),
  'score_time': array([0.01522112, 0.01221085, 0.01183009, 0.0119369 , 0.01150703,
         0.01140976, 0.01488018, 0.01415277, 0.01583982, 0.01520824]),
  'test_ap': array([0.3304334 , 0.25027536, 0.62916667, 0.65227702, 0.42340405,
         0.41997913, 0.42376907, 0.47643237, 0.39089392, 0.44381462]),
  'train_ap': array([0.99323851, 0.99883856, 0.99080087, 1.        , 1.        ,
         1.        , 0.99848332, 0.99114949, 1.        , 0.99944629]),
  'test_auc': array([0.75430108, 0.6284585 , 0.95376344, 0.93817204, 0.77222777,
         0.81571146, 0.73010753, 0.82954545, 0.81666667, 0.84945652]),
  'train_auc': array([0.99935476, 0.99993408, 0.99929023, 1.        , 1.        ,
         1.        , 0.99990521, 0.99920897, 1.        , 0.99996782])},
 0.4440445597443965,
 0.8088410461674836)

In [38]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

kbest = SelectKBest(score_func=f_regression, k=1000)
pipe = Pipeline([('kbest', kbest), ('est', XGBRegressor())])

estimator = BinarizeTargetRegressor(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)
#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([4.97053409, 4.97858119, 4.90172911, 4.88941002, 4.59822679,
         4.57459617, 4.3648901 , 4.36587214, 4.27821112, 4.31290007]),
  'score_time': array([0.02421498, 0.02790785, 0.0195179 , 0.01942205, 0.0190022 ,
         0.0209837 , 0.02204108, 0.02169394, 0.02067685, 0.02723694]),
  'test_ap': array([0.54877275, 0.61441327, 0.57523146, 0.74403337, 0.29733176,
         0.55514536, 0.44959489, 0.48988186, 0.37994174, 0.59819623]),
  'train_ap': array([0.99678199, 1.        , 0.99702381, 0.99790933, 0.99774214,
         0.9988244 , 0.99848332, 0.99774214, 0.99891775, 0.99777014]),
  'test_auc': array([0.9172043 , 0.86462451, 0.90322581, 0.94086022, 0.75924076,
         0.89822134, 0.86021505, 0.84387352, 0.80107527, 0.90652174]),
  'train_auc': array([0.99961285, 1.        , 0.99961285, 0.99974722, 0.99973703,
         0.99986816, 0.99981041, 0.99973632, 0.99987095, 0.9997426 ])},
 0.5252542707185399,
 0.8695062511121417)

In [40]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

kbest = SelectKBest(score_func=f_classif, k=1000)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier())])

estimator = BinarizeTargetClassifier(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)
#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([3.88347125, 3.86313105, 3.92658401, 3.96260095, 4.60677791,
         4.71208   , 4.31603193, 4.33362293, 4.00961304, 3.98262119]),
  'score_time': array([0.02687788, 0.02239895, 0.02731895, 0.02621579, 0.02280498,
         0.03239489, 0.04690909, 0.02360296, 0.02505708, 0.02133298]),
  'test_ap': array([0.45814119, 0.5114899 , 0.55245726, 0.55809398, 0.3815684 ,
         0.51806014, 0.34019864, 0.55218091, 0.5666031 , 0.59025641]),
  'train_ap': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  'test_auc': array([0.89677419, 0.85375494, 0.94086022, 0.89892473, 0.77522478,
         0.87747036, 0.8311828 , 0.88932806, 0.86236559, 0.9076087 ]),
  'train_auc': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])},
 0.50290499368953,
 0.8733494357442465)

In [37]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

kbest = SelectKBest(score_func=f_regression, k=2000)
pipe = Pipeline([('kbest', kbest), ('est', XGBRegressor())])

estimator = BinarizeTargetRegressor(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)
#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([ 8.15743995,  8.12210274,  8.42360973,  8.42132092,  9.37098122,
          9.31548595, 10.27658296, 10.31277704,  8.92877507,  8.93302298]),
  'score_time': array([0.030797  , 0.02911925, 0.02495313, 0.02836418, 0.03121901,
         0.03239703, 0.02920508, 0.03099918, 0.03284502, 0.03219604]),
  'test_ap': array([0.58324912, 0.53678832, 0.55733718, 0.80465201, 0.32463216,
         0.52190954, 0.42623423, 0.53465541, 0.38317005, 0.48406438]),
  'train_ap': array([0.9978458 , 0.99941928, 0.99891775, 0.99947146, 1.        ,
         0.9988244 , 1.        , 0.9988244 , 0.9988794 , 0.9978458 ]),
  'test_auc': array([0.93010753, 0.85573123, 0.90967742, 0.95913978, 0.78121878,
         0.90810277, 0.85913978, 0.87944664, 0.85268817, 0.86956522]),
  'train_auc': array([0.9997419 , 0.99993408, 0.99987095, 0.9999368 , 1.        ,
         0.99986816, 1.        , 0.99986816, 0.99987095, 0.9997426 ])},
 0.515669240154154,
 0.8804817319193194)

# Linear model

In [32]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = SelectKBest(score_func=f_classif, k=1000)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier(booster='gblinear', n_estimators=500))])

estimator = BinarizeTargetClassifier(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([6.60066986, 6.63283086, 6.49263811, 6.48225093, 6.4956181 ,
         6.48249292, 6.56343484, 6.55314302, 6.26921391, 6.43166518,
         6.40357471, 6.40047097, 6.59058809, 6.63607883, 6.36317015,
         6.33034992, 6.43589687, 6.39205885, 6.34181714, 6.30129695]),
  'score_time': array([0.0223031 , 0.02571917, 0.0317328 , 0.03498983, 0.03819871,
         0.03007889, 0.023844  , 0.02570391, 0.03197908, 0.02870679,
         0.02307129, 0.02400899, 0.02163696, 0.02567315, 0.02243876,
         0.02521014, 0.02209115, 0.02492785, 0.02099991, 0.02256894]),
  'test_ap': array([0.44193018, 0.55661422, 0.59741939, 0.42474315, 0.63098004,
         0.55550699, 0.62807073, 0.41379031, 0.59692469, 0.49883274,
         0.34518184, 0.70939626, 0.49854302, 0.61674943, 0.37770613,
         0.67234016, 0.43395722, 0.53905038, 0.48547464, 0.49147543]),
  'train_ap': array([0.8630866 , 0.85057104, 0.87755739, 0.86818676, 0.88683073,
         0.85169862, 0.88682363, 0.92427101, 0.8

In [39]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = SelectKBest(score_func=f_regression, k=1000)
pipe = Pipeline([('kbest', kbest), ('est', XGBRegressor(booster='gblinear', n_estimators=500))])

estimator = BinarizeTargetRegressor(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([7.50556087, 7.54798484, 6.74529886, 6.755445  , 8.20363593,
         8.19933176, 6.382447  , 6.35973787, 8.17302012, 8.21934485,
         6.39117098, 6.37869096, 7.73822713, 7.72626829, 6.38341594,
         6.79504466, 6.34766603, 6.34516621, 7.05043101, 7.23502994]),
  'score_time': array([0.01491404, 0.01900911, 0.01721025, 0.02938509, 0.01638818,
         0.02898812, 0.01462078, 0.01784921, 0.01534224, 0.01802015,
         0.01481199, 0.01534224, 0.01453996, 0.01660085, 0.01416707,
         0.01726723, 0.02060509, 0.01640701, 0.01654911, 0.02567887]),
  'test_ap': array([0.35453143, 0.51151099, 0.45536034, 0.57889723, 0.71392126,
         0.65163744, 0.66683599, 0.35409268, 0.49953868, 0.40618916,
         0.38757683, 0.60477887, 0.42740637, 0.56372031, 0.55215945,
         0.55944689, 0.37088776, 0.48256834, 0.55812302, 0.55370337]),
  'train_ap': array([0.92252892, 0.92622455, 0.94427042, 0.93047151, 0.91739187,
         0.90492821, 0.91689213, 0.95358961, 0.9

# dart model

In [41]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = SelectKBest(score_func=f_classif, k=10000)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier(booster='dart'))])

estimator = BinarizeTargetClassifier(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([32.47808886, 32.54982471, 33.92387724, 33.71080017, 37.04277182,
         36.97403789, 34.23571491, 33.82301474, 33.15622687, 33.00017786,
         33.56296396, 33.92014694, 32.13680291, 32.14086103, 30.41555977,
         30.29273582, 35.10035729, 34.38047075, 31.00805783, 31.52836299]),
  'score_time': array([0.12175417, 0.10957313, 0.13498092, 0.12453389, 0.0897181 ,
         0.09568119, 0.09659195, 0.09276438, 0.09210515, 0.09166908,
         0.09498596, 0.09987116, 0.12884021, 0.11431193, 0.11108518,
         0.10648704, 0.10722494, 0.10810232, 0.09388304, 0.08646202]),
  'test_ap': array([0.43139746, 0.62076003, 0.50080489, 0.31338667, 0.41038866,
         0.68678436, 0.57278123, 0.344155  , 0.4750741 , 0.33545393,
         0.36322842, 0.6461552 , 0.48650999, 0.57472599, 0.39803962,
         0.43118789, 0.42379321, 0.40163702, 0.4626482 , 0.41510175]),
  'train_ap': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.]

In [42]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = SelectKBest(score_func=f_regression, k=10000)
pipe = Pipeline([('kbest', kbest), ('est', XGBRegressor(booster='dart'))])

estimator = BinarizeTargetRegressor(pipe)

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([34.76281714, 34.92209482, 32.82496691, 33.51125097, 33.7081778 ,
         33.13012385, 34.125139  , 34.22302222, 33.48038077, 33.67224216,
         35.67039418, 35.63898396, 38.01090503, 38.14788508, 36.82835102,
         36.7749362 , 35.27873182, 35.48876786, 38.11414504, 38.47436786]),
  'score_time': array([0.08606577, 0.09607625, 0.08476686, 0.11976218, 0.08284521,
         0.09250402, 0.09210706, 0.09936786, 0.08448696, 0.09123778,
         0.08818793, 0.18157601, 0.12030292, 0.1190691 , 0.08283377,
         0.11810684, 0.09230328, 0.13756609, 0.09503102, 0.08324099]),
  'test_ap': array([0.40611154, 0.47565687, 0.53113027, 0.49933435, 0.57004221,
         0.62536942, 0.61356039, 0.36748325, 0.57831474, 0.37112455,
         0.46721334, 0.45694496, 0.48236086, 0.4761322 , 0.61928475,
         0.40925456, 0.46036138, 0.48004914, 0.40194734, 0.31565351]),
  'train_ap': array([0.99944629, 1.        , 0.99941928, 1.        , 1.        ,
         0.9988794 , 0.99944

# stacking

In [46]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = BinarizeTargetTransformer(SelectKBest(score_func=f_classif, k=1000))

li_xgbc = XGBClassifier(booster='gblinear')

cv3 = OrderedKFold(n_splits=10)
stacking = StackingCVClassifier(classifiers=[li_xgbc], meta_classifier=li_xgbc, cv=cv3, use_features_in_secondary=True)

pipe = Pipeline([('kbest', kbest), ('est', BinarizeTargetClassifier(stacking))])

estimator = pipe

cv2 = RepeatedOrderedKFold(n_splits=5, n_repeats=4, random_state=999)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([12.54833794, 12.58112788, 13.10758305, 13.1383884 , 13.106776  ,
         13.1179378 , 13.45734501, 13.53178596, 13.07716012, 13.20949984,
         12.94793224, 12.79163384, 13.22749114, 13.16528797, 12.85756803,
         12.91147971, 12.93086004, 12.86085296, 13.03829098, 13.09941816]),
  'score_time': array([0.02394009, 0.03067017, 0.025172  , 0.04628778, 0.02965903,
         0.02864504, 0.02503991, 0.02274585, 0.02988076, 0.0301621 ,
         0.02555299, 0.02382016, 0.02342296, 0.02629805, 0.02402997,
         0.02741218, 0.02363706, 0.02850485, 0.02806687, 0.02226901]),
  'test_ap': array([0.3793897 , 0.58487179, 0.57939615, 0.46101736, 0.62595626,
         0.60010101, 0.61212376, 0.41788913, 0.59552118, 0.48641452,
         0.35624928, 0.68598536, 0.48594206, 0.6174554 , 0.37220724,
         0.68874118, 0.41791737, 0.53905038, 0.48695963, 0.43620987]),
  'train_ap': array([0.86304774, 0.84712576, 0.88308257, 0.86545179, 0.89020968,
         0.85712452, 0.88554